# Plot Pixel Transforms and Lightcurves
## And look for errors or fringing


In [ ]:
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import starry
from copy import deepcopy
import astropy.units as u
import astropy.constants as const
from astropy.table import Table
from astropy.io import fits, ascii
from corner import corner
import os
import hotspot_fitter
import warnings
import pymc3 as pm

starry.config.lazy = False
starry.config.quiet = True

import theano.tensor as tt
import pymc3_ext as pmx

Run this in a newer starry so that you can get pixel transforms

In [ ]:
assert (starry.__version__ >= "1.2")

## Set up Forward Model

Read in the original map

In [ ]:
t_orig = ascii.read('sim_data/y_original.csv')
y_orig = np.array(t_orig['y'])

In [ ]:
#map1 = starry.Map(4)
map1 = starry.Map(3)

#map1.load("sim_data/earth.jpg", sigma=0.05)
map1[1:,:] = y_orig[1:]
map1.amp = 10**-3
y_input = deepcopy(map1.y[1:])

fig, ax = plt.subplots()
map1.show(projection="rect",colorbar=True,ax=ax)
#fig.savefig('plots/forward_model/forward_map_rect.png',dpi=150,bbox_inches='tight')



In [ ]:
fig, ax = plt.subplots()
map1.show(projection='ortho',ax=ax,colorbar=True)

# Pixel Sampling

Find the pixel transforms

Check if we can get the lightcurve from the design matrix

In [ ]:
lat_t, lon_t, Y2P, P2Y, Dx, Dy = map1.get_pixel_transforms(oversample=2)
npix = lat_t.shape[0]

In [ ]:
# plt.plot(x,sys.flux(x))
# f_from_matrix = X_inf.dot(y_orig)* 10**log_amp + 1.
# plt.plot(x,f_from_matrix)

## How do the transforms from individual pixels look?

In [ ]:
def plot_pixels(pixels):
    cm = plt.cm.get_cmap('plasma')
    plt.scatter(lon_t, lat_t,c=pixels,cmap=cm,marker='s',s=500)
    plt.xlabel("Longitude (deg)")
    plt.ylabel("Latitude (deg)")

def plot_pixels_and_sph_harmonic_map(pixelInd=6,returnY=False):
    p_test = np.zeros(npix)
    p_test[pixelInd] = 0.999 * map1.amp
    
    plot_pixels(p_test)


    ytest = np.dot(P2Y,p_test)
    map2 = map1 #deepcopy(map1)
    map2.amp = 10**-3
    planetY = ytest[1:] / map1.amp
    map2[1:, :] = planetY

    map2.show(projection="rectangular")
    
    if returnY == True:
        return deepcopy(planetY)

In [ ]:
y_mid_s = plot_pixels_and_sph_harmonic_map(pixelInd=6,returnY=True)

In [ ]:
y_eq = plot_pixels_and_sph_harmonic_map(pixelInd=14,returnY=True)

Single pixels transformed to spherical harmonic coefficients don't seem so bad (Jacob and I were a little worried about ringing and errors in the transform)

## How does the Inverse Transform of the original forward map look?

Original forward map

In [ ]:
map1[1:,:] = y_orig[1:]
fig, ax = plt.subplots()
map1.show(projection="rect",colorbar=True,ax=ax)
#fig.savefig('plots/forward_model/forward_map_rect.png',dpi=150,bbox_inches='tight')


In [ ]:
y_input_full = np.append(1,y_input)
px_input = np.dot(Y2P,y_input_full)

In [ ]:
plot_pixels(px_input)

## Diffent Exploration: How much do the poles matter?
Megan and I were wondering whether the poles are intrinsically hard because they don't affect the lightcurve much?

### First, plot the pixel at the pole

In [ ]:
y_s_pole = plot_pixels_and_sph_harmonic_map(pixelInd=0,returnY=True)


With pixel sampling, one pixel covers the whole pole it seems

In [ ]:
y_n_pole = plot_pixels_and_sph_harmonic_map(pixelInd=29,returnY=True)



In [ ]:
def make_system(planetY):
    up_bin = 10 ## up-bin for clarity
    lc_precision = 145e-6 / np.sqrt(up_bin) ## measured from broadband F444W precision
    cadence = 2.72486 * up_bin ## between exposures
    npoints = int(np.round(0.2 * 24. * 3600.  / (cadence))) ## duration/cadence

    modelDat = ascii.read('sim_data/hd189733b_spec_model_binned.csv')
    modelDat[1]['depth']

    M_star = 0.812 ## Addison et al. 2019, solar masses
    R_star = 0.765 ## Addison et al. 2019, solar radii
    inc=85.69 ## Addison et al. 2019
    #inc=90.0
    rp = 0.1504 * R_star ## Addison et al. 2019, Solar radii
    P_b = 2.218577 ## days
    log_amp = np.log10(modelDat[1]['depth'])
    t0 = 0.0

    M_planet = (1.166 * u.Mjup).to(u.Msun).value ## Addison et al. 2019, Solar masses

    prot_star = 1.0

    x = np.linspace(0.5 * P_b - 0.1,0.5 * P_b + 0.1,npoints)
    #x = np.linspace(0.,P_b,4096)

    A = starry.Primary(starry.Map(ydeg=0, udeg=2, amp=1.0), m=M_star, r=R_star,
                       prot=prot_star )
    planetMap = starry.Map(3)
    planetMap[1:,:] = planetY
    b = starry.kepler.Secondary(planetMap,
                                m=M_planet,r=rp,prot=P_b,porb=P_b,t0=t0,inc=inc)
    b.map.amp = 10**log_amp


    b.theta0 = 180.0 + 0.0
    sys = starry.System(A,b)
    
    return x,sys

In [ ]:
x, sys_s_pole = make_system(y_s_pole)
x, sys_n_pole = make_system(y_n_pole)
x, sys_eq = make_system(y_eq)
x, sys_mid = make_system(y_mid_s)



In [ ]:
plt.plot(x,sys_s_pole.flux(x),label='S Pole')
plt.plot(x,sys_n_pole.flux(x),label='N Pole')
plt.plot(x,sys_eq.flux(x),label='Equator')
plt.plot(x,sys_mid.flux(x),label='S Mid-latitude')
plt.legend()
plt.xlabel("Time (days)")
plt.ylabel("Flux")
plt.savefig('plots/pix_samp_lc/lc_of_example_pixels.pdf')

In [ ]:
sys_n_pole.show(t=np.linspace(1.0,1.2,40),figsize=(12,12))

In [ ]:
sys_s_pole.show(t=np.linspace(1.0,1.2,40),figsize=(12,12))